In [1]:
import os
import copy
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
xlsx_file = "./SlidesInfo.xlsx"
df_all = pd.read_excel(xlsx_file)
df_all.sample(5)

,文件名,病理号,路径,朱诊断,医生诊断,片源,制式,Unnamed: 7
9007,TC18051344,TC18051344,DATA/0TIFF/NFYY+HY/HSIL,NaN,HSIL,华银,沉降式,NaN
12262,TC17052210,TC17052210,DATA/0TIFF/NFYY+HY/ASCUS,NaN,ASCUS,华银,沉降式,NaN
8945,TC17012459,TC17012459,DATA/0TIFF/NFYY+HY/HSIL,NaN,HSIL,华银,沉降式,NaN
956,TC18050331,TC18050331,DATA/0TIFF/NFYY+HY/normal,NORMAL,NORMAL,华银,沉降式,NaN
12845,TC18014386,TC18014386,DATA/0TIFF/NFYY+HY/ASCUS,NaN,ASCUS,华银,沉降式,NaN


In [3]:
df = df_all[pd.notnull(df_all["朱诊断"])]
df.sample(5)

,文件名,病理号,路径,朱诊断,医生诊断,片源,制式,Unnamed: 7
2304,TC18049099,TC18049099,DATA/0TIFF/NFYY+HY/normal,NORMAL,NORMAL,华银,沉降式,NaN
1229,TC18050849,TC18050849,DATA/0TIFF/NFYY+HY/normal,NORMAL,NORMAL,华银,沉降式,NaN
5955,TC18052901,TC18052901,DATA/0TIFF/NFYY+HY/normal,NORMAL,NORMAL,华银,沉降式,NaN
4263,TC18051707,TC18051707,DATA/0TIFF/NFYY+HY/normal,NORMAL,NORMAL,华银,沉降式,NaN
8486,TC16009468,TC16009468,DATA/0TIFF/NFYY+HY/HSIL,SCC,AGC,华银,沉降式,NaN


In [4]:
print(df_all.shape, df.shape)

(16460, 8) (6645, 8)


In [5]:
label_counts_all = df_all["医生诊断"].value_counts()
print(label_counts_all)

NORMAL              6066
ASCUS               5048
LSIL                1298
HSIL                1160
ASCH                 895
FUNGI                513
AGC                  238
CC                   209
TRI                  187
ACTINO               133
ASCUS+FUNGI           84
EC                    81
ASCUS+CC              77
SCC                   66
unknown               37
VIRUS                 36
ADC                   31
ASCUS+TRI             27
HSIL                  20
AGC2                  19
LSIL+ASCH             17
ASCUS                 16
HISL                  16
AGC1                  13
ASCH+FUNGI            12
LSIL+FUNGI            12
LSIL+CC               10
ASCUS+ACTINO           8
ASCH                   8
ASUCS                  4
                    ... 
HSIL+TRI               1
ASCH+AGC               1
DHSIC                  1
AGC(zigongneimo)       1
ASCUS+FUNGI+CC         1
HISL+ACTINO            1
unclear                1
unsatisfied+less       1
HSIL+CA                1


In [6]:
label_counts = df["朱诊断"].value_counts()
print(label_counts)

NORMAL          5454
LSIL             261
ASCUS            205
HSIL             186
FUNGI            143
ASCH              66
SCC               63
CC                44
EC                35
ACTINO            32
TRI               32
AGC               31
LSIL+ASCH         24
ASCUS+CC          14
LSIL+CC           11
VIRUS              8
ASCH+CC            4
TRI+FUNGI          3
ACTINO+CC          3
AGC1               2
ASCH+VIRUS         2
ACTINO+FUNGI       2
LSIL+FUNGI         2
LSIL+EC            2
ASCUS+FUNGI        2
HSIL+FUNGI         1
改成球菌               1
maybe_NORMAL       1
ASCUS+CC+TRI       1
maybe_normal       1
内膜角质               1
LSIL+EC+ASCH       1
FUNGI+TRI          1
ASCUS?             1
ASCH+FUNGI         1
ADC                1
TRI+VIRUS          1
AGC2               1
TRI+ACTINO         1
Name: 朱诊断, dtype: int64


In [7]:
files = {}
for i, row in df_all.iterrows():
    hasZhu = not pd.isnull(row["朱诊断"])
    files[str(row["文件名"])] = {"hasZhu":hasZhu, "label":str(row["朱诊断"]) if hasZhu else str(row["医生诊断"]), "path":str(row["路径"]), "isChosen":False}

In [8]:
def smart_select(files_orig, choices):
    """ Sampling files based on choices. 
    If given class is specified with number, then randomly choose files from data server. 
    Or else, choose all unselected files.
    Files diagnozed by Zhu are choosed first.
    :param files_orig: file entry with filename, label, and path
    :param choices: {"NORMAL":xxx, "ASCUS":xxx}
    """
    files = copy.deepcopy(files_orig)
    chosen = {}
    for class_i,number in choices.items():
        chosen[class_i] = []
        number = len(files) if number == "all" else min(number, len(files))
        
        files_zhu = {}
        files_else = {}
        for key,value in files.items():
            if not value["isChosen"] and class_i in value["label"]:
                if value["hasZhu"]:
                    files_zhu[key] = value
                else:
                    files_else[key] = value
        
        count_zhu = 0
        for key,value in files_zhu.items():
            if count_zhu < number:
                chosen[class_i].append([key,value])
                files[key]["isChosen"] = True
                count_zhu += 1
            else:
                break
        count_else = 0
        for key,value in files_else.items():
            if count_zhu+count_else < number:
                chosen[class_i].append([key,value])
                files[key]["isChosen"] = True
                count_else += 1
            else:
                break
        print("{}: zhu {} other {}".format(class_i.ljust(10), str(count_zhu).ljust(6), count_else))
        
    return chosen


def pprint(chosen):
    for key in chosen:
        print("{}: {}".format(key, len(chosen[key])))

In [9]:
choices = {"NORMAL":3000, "ASCUS":500, "LSIL":1000, "ASCH":500, "HSIL":1000, "SCC":66, 
           "AGC":238, "FUNGI":500, "CC":209, "TRI":187, "ACTINO":133, "EC":81, "VIRUS":40, 
           "AGC1":13, "ADC":31, "AGC2":19}

chosen = smart_select(files, choices)

SCC       : zhu 63     other 3
ASCH      : zhu 98     other 402
ACTINO    : zhu 38     other 95
AGC1      : zhu 2      other 9
TRI       : zhu 38     other 149
VIRUS     : zhu 8      other 29
NORMAL    : zhu 3000   other 0
AGC2      : zhu 1      other 12
HSIL      : zhu 187    other 813
FUNGI     : zhu 147    other 353
ASCUS     : zhu 220    other 280
ADC       : zhu 1      other 29
CC        : zhu 55     other 154
LSIL      : zhu 263    other 737
AGC       : zhu 31     other 207
EC        : zhu 35     other 46


In [10]:
print(chosen["ADC"])

[['TC18050298', {'hasZhu': True, 'isChosen': True, 'label': 'ADC', 'path': 'DATA/0TIFF/NFYY+HY/AGC'}], ['XB1800796', {'hasZhu': False, 'isChosen': True, 'label': 'ADC', 'path': 'DATA/0TIFF/NFYY+HY/AGC'}], ['BD1605204', {'hasZhu': False, 'isChosen': True, 'label': 'ADC', 'path': 'DATA/0TIFF/NFYY+HY/ADC'}], ['BD1601497', {'hasZhu': False, 'isChosen': True, 'label': 'ADC', 'path': 'DATA/0TIFF/NFYY+HY/ADC'}], ['BD1502535', {'hasZhu': False, 'isChosen': True, 'label': 'ADC', 'path': 'DATA/0TIFF/NFYY+HY/AGC'}], ['BD1601533', {'hasZhu': False, 'isChosen': True, 'label': 'ADC', 'path': 'DATA/0TIFF/NFYY+HY/ADC'}], ['BD1603609', {'hasZhu': False, 'isChosen': True, 'label': 'ADC', 'path': 'DATA/0TIFF/NFYY+HY/ADC'}], ['L15-23667', {'hasZhu': False, 'isChosen': True, 'label': 'ADC', 'path': 'DATA/0TIFF/ZHENGZHOU/abnormal_40x'}], ['1704689', {'hasZhu': False, 'isChosen': True, 'label': 'ADC', 'path': 'DATA/0TIFF/NFYY+HY/ADC'}], ['1712933', {'hasZhu': False, 'isChosen': True, 'label': 'ADC', 'path': 

In [11]:
# save chosen to file
import pickle

with open("chosen.pkl", 'wb') as f:
    pickle.dump(chosen, f)

In [12]:
# open saved chosen file
with open("chosen.pkl", 'rb') as f:
    chosen_ = pickle.load(f)

print(chosen_.keys())

def get_label(chosen, basename):
    for class_i,entries in chosen.items():
        for entry in entries:
            if entry[0] == basename:
                return entry[1]["label"].split('+')  # list of labels
    return None

dict_keys(['SCC', 'ASCH', 'ACTINO', 'AGC1', 'TRI', 'NORMAL', 'LSIL', 'AGC2', 'HSIL', 'FUNGI', 'ASCUS', 'ADC', 'CC', 'VIRUS', 'AGC', 'EC'])


In [13]:
# goto xiongxin 4k
xiongxin_4k = []
subclasses = {"NORMAL":3000}
for class_i in subclasses:
    subfiles = chosen[class_i]
    for key,value in subfiles:
#         file_path = os.path.join("/home/data_samba", value["path"], key)
#         if os.path.isfile(file_path+".kfb"):
#             file_path += ".kfb"
#         elif os.path.isfile(file_path+".tif"):
#             file_path += ".tif"
#         else:
#             continue # pass
        xiongxin_4k.append(key)
        
with open("xiongxin_4k.txt", 'w') as f:
    for file_path in xiongxin_4k:
        f.write(file_path+'\n')

In [14]:
# goto yaozhong 6k
yaozhong_6k = []
subclasses = {"ASCUS":500, "LSIL":1000, "ASCH":500, "HSIL":1000, "SCC":66, 
           "AGC":238, "FUNGI":500, "CC":209, "TRI":187, "ACTINO":133, "EC":81, "VIRUS":40, 
           "AGC1":13, "ADC":31, "AGC2":19}
for class_i in subclasses:
    subfiles = chosen[class_i]
    for key,value in subfiles:
#         file_path = os.path.join("/home/data_samba", value["path"], key)
#         if os.path.isfile(file_path+".kfb"):
#             file_path += ".kfb"
#         elif os.path.isfile(file_path+".tif"):
#             file_path += ".tif"
#         else:
#             continue # pass
        yaozhong_6k.append(key)
        
with open("yaozhong_6k.txt", 'w') as f:
    for file_path in yaozhong_6k:
        f.write(file_path+'\n')

In [ ]:
# get full path by basename, read basename from txt
basenames = []
with open("yaozhong_6k.txt", 'r') as f:
    for line in f.readlines():
        basenames.append(line.strip())
        
def scan_files(directory, prefix=None, postfix=None):
    files_list = []
    for root, sub_dirs, files in os.walk(directory):
        for special_file in files:
            if postfix:
                if special_file.endswith(postfix):
                    files_list.append(os.path.join(root, special_file))
            elif prefix:
                if special_file.startswith(prefix):
                    files_list.append(os.path.join(root, special_file))
            else:
                files_list.append(os.path.join(root, special_file))
    return files_list

yaozhong_6k = []
all_files = scan_files("/mnt/DATA", postfix=".kfb") + scan_files("/mnt/DATA", postfix=".tif")
for basename in basenames:
    for file in all_files:
        if basename in file:
            yaozhong_6k.append(file)
            break

with open("yaozhong_6k_fullpath.txt", 'w') as f:
    for file_path in yaozhong_6k:
        f.write(file_path+'\n')